In [1]:
import os
import numpy
from pathlib import Path
import re

# ==========================================
# 🚑 3D SCANNER REPAIR & RUN (Final Fix)
# ==========================================

print("🚑 1. กำลังกู้ชีพ Numpy (Undo File Patch)...")
try:
    # หาไฟล์ stride_tricks.py ตัวปัญหา
    stride_tricks_path = Path(numpy.__file__).parent / "lib" / "stride_tricks.py"
    if stride_tricks_path.exists():
        with open(stride_tricks_path, "r") as f:
            lines = f.readlines()
        
        # กรองเอาบรรทัดที่เราเคยเติมไปออกให้หมด
        clean_lines = [line for line in lines if "KAGGLE HOTFIX" not in line and "from numpy import broadcast_to" not in line]
        
        if len(clean_lines) != len(lines):
            with open(stride_tricks_path, "w") as f:
                f.writelines(clean_lines)
            print("✅ ลบโค้ดงูกินหางออกแล้ว Numpy กลับมาปกติ!")
        else:
            print("✅ ไฟล์ปกติดี ไม่ต้องแก้")
except Exception as e:
    print(f"⚠️ Warning: กู้ไฟล์ไม่สำเร็จ ({e}) แต่จะพยายามรันต่อ")

# ---------------------------------------------------------

print("\n📥 2. กำลังโหลดโปรแกรมใหม่...")
if os.path.exists("3DSCAN"):
    !rm -rf 3DSCAN
!git clone https://github.com/PRIDA-TAKON/3DSCAN.git
os.chdir("3DSCAN")

# ---------------------------------------------------------

print("\n🔧 3. กำลังติดตั้ง Runtime Patch (Memory Hack)...")
# เราจะแก้ไฟล์ 3d-scan.py ให้ใช้ Runtime Patch แทน File Patch
target_script = "3d-scan.py"
if not os.path.exists(target_script):
    import glob
    target_script = glob.glob("*.py")[0]

with open(target_script, "r") as f:
    code = f.read()

# โค้ด Patch แบบใหม่ (แก้ใน RAM เท่านั้น)
runtime_patch_code = """
def apply_runtime_fix():
    print("🩹 Applying Runtime Memory Patch...")
    try:
        import numpy
        import numpy.lib.stride_tricks
        # ยัดเยียด broadcast_to เข้าไปใน stride_tricks ใน Memory
        if not hasattr(numpy.lib.stride_tricks, "broadcast_to"):
            numpy.lib.stride_tricks.broadcast_to = numpy.broadcast_to
            print("✅ Runtime patch successful: broadcast_to is available.")
    except Exception as e:
        print(f"⚠️ Runtime patch failed: {e}")
"""

# เปลี่ยนคำสั่งเรียกใช้ฟังก์ชัน
code = code.replace("patch_numpy()", "apply_runtime_fix()")
# ปิด GPU Matching เพื่อความชัวร์
code = code.replace('"--SiftMatching.use_gpu", "1"', '"--SiftMatching.use_gpu", "0"')
# เติมฟังก์ชันใหม่ลงไปท้ายไฟล์
code += "\n\n" + runtime_patch_code

with open(target_script, "w") as f:
    f.write(code)
print("✅ อัปเกรดโค้ดเสร็จสมบูรณ์!")

# ---------------------------------------------------------

print(f"\n🚀 4. เริ่มรัน {target_script} ...")
print("-" * 50)
!python "{target_script}" 2>&1 | tee 3d-scan.log.txt

🚑 1. กำลังกู้ชีพ Numpy (Undo File Patch)...
✅ ไฟล์ปกติดี ไม่ต้องแก้

📥 2. กำลังโหลดโปรแกรมใหม่...
Cloning into '3DSCAN'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 60 (delta 10), reused 5 (delta 3), pack-reused 30 (from 1)
Receiving objects: 100% (60/60), 382.58 KiB | 3.86 MiB/s, done.
Resolving deltas: 100% (20/20), done.

🔧 3. กำลังติดตั้ง Runtime Patch (Memory Hack)...
✅ อัปเกรดโค้ดเสร็จสมบูรณ์!

🚀 4. เริ่มรัน 3d-scan.py ...
--------------------------------------------------
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.4 re